In [1]:
import os
import pandas as pd

In [80]:
experiment_df = pd.read_csv("output/all-profile-metadata.csv")
experiment_df

,Vendor,Batch,Plate_Map_Name,Assay_Plate_Barcode,Modality,Images_per_well,Sites-SubSampled,Binning,Magnification,Number_of_channels,z_plane,BF_Zplanes,spinning-disc,aperture,dry-immersion,vs-brightfield,simultaneous-excitation,sites
0,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_4site_10XPA,Confocal,4,NaN,1,10,6,1,NaN,no,0.45,dry,NaN,NaN,4
1,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crest,Confocal,4,NaN,1,10,6,1,NaN,yes,0.45,dry,NaN,NaN,4
2,MolDev,Scope1_MolDev_10X_4siteZ,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crestz,Confocal,4,NaN,1,10,6,3,NaN,yes,0.45,dry,NaN,NaN,4
3,MolDev,Scope1_MolDev_20X_4site,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_20XPA_Crestz,Confocal,4,NaN,1,20,6,3,NaN,yes,0.75,dry,NaN,NaN,4
4,MolDev,Scope1_MolDev_20X_9site,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_9site_20XPA,Confocal,9,NaN,1,20,6,1,NaN,no,0.75,dry,NaN,NaN,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_5Ch,JUMP-MOA_compound_platemap,BRO0117056_20x,Confocal,9,4.0,1,20,5,12,13,NaN,1.00,water,NaN,NaN,4
180,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_5Ch_12Z,JUMP-MOA_compound_platemap,BRO0117056_20xb,Confocal,9,4.0,1,20,5,12,15,NaN,1.00,water,NaN,NaN,4
181,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO0117059_20X,Confocal,9,4.0,1,20,6,12,15,NaN,1.00,water,NaN,NaN,4
182,Yokogawa_US,4siteSubSample_Scope1_Yokogawa_US_20X_6Ch_BRO0...,JUMP-MOA_compound_platemap,BRO01177034_20x,Confocal,9,4.0,1,20,6,12,15,NaN,1.00,water,NaN,NaN,4


In [97]:

def same_test(experiment_metadata, profile_parent_dir1, profile_parent_dir2, batch_col="Batch"):
    """
    For two divergent profiles, check what is the same and what has changed. 

    batch_col is the name of the column to distinguish the profile parent folder. Eg. "Scope1_MolDev_10X" or "1siteSubSample_Scope1_MolDev_10X"
    Output df will also use this batch_col name
    """

    for ind, a_vendor in enumerate(experiment_metadata["Vendor"].unique()):
        print(f"Processing {a_vendor}")
        vendor_data = experiment_metadata.loc[experiment_metadata["Vendor"] == a_vendor]
        for a_batch in vendor_data[batch_col].unique():
            batch_data = vendor_data.loc[vendor_data[batch_col] == a_batch]
            for a_plate in batch_data["Assay_Plate_Barcode"].unique():
                # plate_data = batch_data.loc[batch_data["Assay_Plate_Barcode"] == a_plate]
                # data_path1 = os.path.join(profile_parent_dir1, a_batch, a_plate, a_plate+"_normalized_feature_select_negcon_batch.csv.gz")
                # data_path2 = os.path.join(profile_parent_dir2, a_batch, a_plate, a_plate+"_normalized_feature_select_negcon_batch.csv.gz")
                data_path1 = os.path.join(profile_parent_dir1, a_batch, a_plate, a_plate+".csv.gz")
                data_path2 = os.path.join(profile_parent_dir2, a_batch, a_plate, a_plate+".csv.gz")
                load_data1 = pd.read_csv(data_path1)
                load_data2 = pd.read_csv(data_path2)
                # diff_cols = set(load_data1.columns) - set(load_data2.columns)
                # diff_cols = set(load_data2.columns) - set(load_data1.columns)
                diff_cols = load_data1.columns.symmetric_difference(load_data2.columns)
                diff_cols = {i for i in load_data2.columns if "Image" not in i}
                same_cols = load_data1.columns.intersection(load_data2.columns)
                same_cols = {i for i in load_data2.columns if "Metadata" in i}
                print(same_cols)

same_test(experiment_df[experiment_df["Assay_Plate_Barcode"] == "CP_Broad_Phenix_C_BIN1_1Plane_P1"], "../jump-scope/profiles", "../jump-scope/profiles-with-image-features")


Processing PE
{'Metadata_Plate', 'Metadata_Count_Cells', 'Metadata_Site_Count', 'Metadata_Count_Cytoplasm', 'Metadata_Object_Count', 'Metadata_Count_Nuclei', 'Metadata_Well'}
{'Metadata_Plate', 'Metadata_Count_Cells', 'Metadata_Site_Count', 'Metadata_Count_Cytoplasm', 'Metadata_Object_Count', 'Metadata_Count_Nuclei', 'Metadata_Well'}
{'Metadata_Plate', 'Metadata_Count_Cells', 'Metadata_Site_Count', 'Metadata_Count_Cytoplasm', 'Metadata_Object_Count', 'Metadata_Count_Nuclei', 'Metadata_Well'}


In [90]:
# Add the metadata from the updated profiles to profiles_original
import glob

def add_metadata(experiment_metadata, original_profiles, updated_profiles):
    """
    Helper function to add the metadata from updated_profiles to original_profiles
    """
    for ind, a_vendor in enumerate(experiment_metadata["Vendor"].unique()):
        print(f"Processing {a_vendor}")
        vendor_data = experiment_metadata.loc[experiment_metadata["Vendor"] == a_vendor]
        for a_batch in vendor_data["Batch"].unique():
            if "siteSub" not in a_batch:
                batch_data = vendor_data.loc[vendor_data["Batch"] == a_batch]
                for a_plate in batch_data["Assay_Plate_Barcode"].unique():
                    print(a_batch, a_plate)
                    orig_paths = glob.glob(os.path.join(original_profiles, a_batch, a_plate, a_plate+"*.csv.gz"))
                    meta_path = os.path.join(updated_profiles, a_batch, a_plate, a_plate+".csv.gz")
                    meta_df = pd.read_csv(meta_path)
                    merge_cols = [i for i in with_md.columns if "Metadata" in i]
                    print(merge_cols)
                    on_col = "Metadata_Well"
                    for orig_csv in orig_paths:
                        orig_df = pd.read_csv(orig_csv)
                        orig_df = orig_df.merge(meta_df[merge_cols], on=on_col)
                        orig_df.to_csv(orig_csv, index=False)

add_metadata(experiment_df, "../jump-scope/profiles", "../jump-scope/profiles-with-image-features")

Processing MolDev
Scope1_MolDev_10X Plate2_PCO_6ch_4site_10XPA
['Metadata_Plate', 'Metadata_Well', 'Metadata_Site_Count', 'Metadata_Count_Cells', 'Metadata_Count_Cytoplasm', 'Metadata_Count_Nuclei', 'Metadata_Object_Count']
Scope1_MolDev_10X Plate3_PCO_6ch_4site_10XPA_Crest
['Metadata_Plate', 'Metadata_Well', 'Metadata_Site_Count', 'Metadata_Count_Cells', 'Metadata_Count_Cytoplasm', 'Metadata_Count_Nuclei', 'Metadata_Object_Count']
Scope1_MolDev_10X_4siteZ Plate3_PCO_6ch_4site_10XPA_Crestz
['Metadata_Plate', 'Metadata_Well', 'Metadata_Site_Count', 'Metadata_Count_Cells', 'Metadata_Count_Cytoplasm', 'Metadata_Count_Nuclei', 'Metadata_Object_Count']
Scope1_MolDev_20X_4site Plate3_PCO_6ch_4site_20XPA_Crestz
['Metadata_Plate', 'Metadata_Well', 'Metadata_Site_Count', 'Metadata_Count_Cells', 'Metadata_Count_Cytoplasm', 'Metadata_Count_Nuclei', 'Metadata_Object_Count']
Scope1_MolDev_20X_9site Plate2_PCO_6ch_9site_20XPA
['Metadata_Plate', 'Metadata_Well', 'Metadata_Site_Count', 'Metadata_Count